### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/finals/baselines
Obtaining file:///project/finals/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
Requirement already up-to-date: tensorflow-gpu==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)


In [2]:
import time
import gym
import random
import numpy as np
import pandas as pd

from datetime import datetime

from tensorflow import keras
from tensorflow.keras import layers

from baselines.ppo2 import ppo2
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from baselines import deepq
from baselines import bench
from baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2966595622020009254,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10596021800190463974
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16719128541956159157
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
%cd ..
logdir = "Training/Logs/DQN/" + datetime.now().strftime("%Y%m%d-%H%M%S")

/project/finals


In [4]:
cicddos2019 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.feather")
cicdos2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-dos2017/clean/cicdos2017.feather")
cicids2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ids2017/clean/cicids2017.feather")
data = pd.concat([cicddos2019], ignore_index=True)
print(len(data.columns))
data.head(5)

69


,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,...,0,8185.583496,28337.111328,98168.0,3.0,9529897.0,351582.62500,10001143.0,9048097.0,Benign
1,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,...,0,35028.417969,121314.914062,420255.0,4.0,9493930.0,351541.09375,9978130.0,8820294.0,Benign
2,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62416.000000,0.000000,62416.0,62416.0,36373056.0,0.00000,36373056.0,36373056.0,Benign
3,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62413.000000,0.000000,62413.0,62413.0,36372292.0,0.00000,36372292.0,36372292.0,Benign
4,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62409.000000,0.000000,62409.0,62409.0,36372216.0,0.00000,36372216.0,36372216.0,Benign


In [5]:
cicdos2017_features = ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"]
cicids2017_features = ["Protocol", "Avg Bwd Segment Size", "Packet Length Max", "Bwd Packet Length Min", "Fwd IAT Mean"]
cicddos2019_features = ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"]
important_features = cicddos2019_features + ["Label", "Timestamp"] # Adding Timestamp for custom environment logic
important_features = list(set(important_features))
print(len(important_features))

7


In [6]:
removable_features = data.columns ^ important_features
print(removable_features.shape)
skinny_data = data.drop(labels=removable_features, axis='columns')

(62,)


In [7]:
skinny_data.info()
skinny_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138007 entries, 0 to 138006
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Timestamp                 138007 non-null  datetime64[ns]
 1   Fwd Packets Length Total  138007 non-null  float32       
 2   Bwd Packet Length Min     138007 non-null  float32       
 3   ACK Flag Count            138007 non-null  int8          
 4   URG Flag Count            138007 non-null  int8          
 5   Down/Up Ratio             138007 non-null  float32       
 6   Label                     138007 non-null  category      
dtypes: category(1), datetime64[ns](1), float32(3), int8(2)
memory usage: 3.0 MB


,Fwd Packets Length Total,Bwd Packet Length Min,ACK Flag Count,URG Flag Count,Down/Up Ratio
count,1.380070e+05,138007.000000,138007.000000,138007.000000,138007.000000
mean,4.018388e+03,25.685892,0.198584,0.275283,0.702986
std,5.148026e+04,52.287216,0.398936,0.446658,0.937449
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.800000e+01,0.000000,0.000000,0.000000,0.000000
50%,8.600000e+01,0.000000,0.000000,0.000000,1.000000
75%,2.064000e+03,46.000000,0.000000,1.000000,1.000000
max,1.526642e+07,1460.000000,1.000000,1.000000,23.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [8]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    cols = df.columns[:-1]
    vc = df[col].value_counts()
    n = vc.iloc[-1]
    m = vc.iloc[0]
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    df = df.drop(index=initial_cut.index)

    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [9]:
skinny_data.drop('Timestamp', inplace=True, axis=1)
print(skinny_data['Label'].value_counts())
skinny_data['Label'] = skinny_data['Label'].astype('object')
atk_idx = skinny_data.loc[skinny_data['Label'] != "Benign"].index
skinny_data.loc[atk_idx, 'Label'] = 1.0
skinny_data.loc[data.index.difference(atk_idx), 'Label'] = 0.0
skinny_data['Label'] = skinny_data['Label'].astype(dtype=np.float32)

x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(skinny_data, train_frac=0.9, val_frac=0.0, test_frac=0.1)

Benign           95095
DrDoS_NTP        13687
TFTP             13027
Syn               6713
UDP               2158
MSSQL             1763
DrDoS_UDP         1174
UDP-lag           1097
DrDoS_MSSQL        927
DrDoS_DNS          552
LDAP               484
DrDoS_SSDP         413
DrDoS_SNMP         392
Portmap            215
DrDoS_LDAP         174
NetBIOS             73
DrDoS_NetBIOS       51
UDPLag              10
WebDDoS              2
Name: Label, dtype: int64


In [10]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


Training
(77240, 5)
(77240,)
Validation
(0, 5)
(0,)
Testing
(8582, 5)
(8582,)


In [11]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_val = np.expand_dims(x_val, -1)

In [12]:
y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)

### Custom environment

In [13]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
        # All the features we have, 67 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(important_features) - 2,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs
        return obs

Train classifier using dqn

In [14]:
def ids_dqn():
    logger.configure(dir=logdir, format_strs=['stdout', 'tensorboard'])
    env = IdsEnv(images_per_episode=4096)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.learn(
        env,
        "mlp",
        total_timesteps=int(1.0e6),
        print_freq=10,    
    )

    model.save('dqn_cicddos2019_topfeat.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = ids_dqn()
print("DQN Training Time:", time.time() - start_time)


Logging to Training/Logs/DQN/20220408-141842






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use keras.layers.flatten instead.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 63       |
| episodes                | 10       |
| mean 100 episode reward | 2.35e+03 |
| steps                   | 3.69e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 23       |
| episodes                | 20       |
| mean 100 episode reward | 2.68e+03 |
| steps                   | 7.78e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 30       |
| mean 100 episode reward | 2.97e+03 |
| steps                   | 1.19e+05 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 40       |
| mean 100 episode reward | 3.14e+03 |
| steps                   | 1.6e+05  |
--------------------------------------
--------------------------------------
| % time spent exploring 

In [15]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=4096, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model(obs[None])[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        FPR = (float(FP) / (TN + FP)) # 1 - specificity
        f1_score = 2 * (precision * recall) / (precision + recall)
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))

ids_eval(dqn_model)

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 82.62795849364579%
Precision: 66.6822103054325%
Recall/TPR/Sensitivity: 97.91167408421774%
FPR: 25.265205091937766%
F1 score: 0.7933425797503467
